# Word2vec
## Existing models
https://github.com/RaRe-Technologies/gensim-data

In [1]:
import gensim.downloader as api

### English model

In [2]:
model_eng = api.load("glove-twitter-25")

In [3]:
model_eng.most_similar("analyst")

[('administrator', 0.9545297026634216),
 ('consultant', 0.9456777572631836),
 ('advisor', 0.9416788816452026),
 ('engineer', 0.9342100620269775),
 ('assistant', 0.9327148795127869),
 ('finance', 0.9319754838943481),
 ('coordinator', 0.9300969839096069),
 ('specialist', 0.9288251399993896),
 ('technical', 0.9264996647834778),
 ('consulting', 0.9245263934135437)]

### Russian model

In [4]:
model_rus = api.load("word2vec-ruscorpora-300")

In [5]:
model_rus.most_similar("аналитик_NOUN")

[('ufg_NOUN', 0.6764760613441467),
 ('j::son_NOUN', 0.6654938459396362),
 ('эксперт_NOUN', 0.664729118347168),
 ('инвестбанк_NOUN', 0.6485406160354614),
 ('надоршин_NOUN', 0.6478258371353149),
 ('дмитрий::лютягин_NOUN', 0.6457507610321045),
 ('economist::intelligence_NOUN', 0.6367553472518921),
 ('брокеркредитсервис_NOUN', 0.6350533962249756),
 ('morgan::stanley_NOUN', 0.632604718208313),
 ('экономист_NOUN', 0.6229816675186157)]

## My corpus

In [18]:
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
import gensim 
from gensim.models import Word2Vec, KeyedVectors 
from tqdm import tqdm_notebook
from pprint import pprint

warnings.filterwarnings(action = 'ignore') 

In [2]:
def tokenize(text):
    punctuations = '!"#$%&\'()*+,./:;<=>?@[\\]^—-_`{|}~‘’«»'
    tokens = word_tokenize(text.lower())
    tokens = list(filter(lambda word: word not in punctuations, tokens))
    return tokens

In [3]:
sample = open("text_perm_it_job_description.txt", "r", encoding='utf') 
f = sample.read().replace("\n", " ") 

data = []
for i in tqdm_notebook(sent_tokenize(f)):
    data.append(tokenize(i))

HBox(children=(IntProgress(value=0, max=5577), HTML(value='')))

## CBOW

In [5]:
%%time
model1 = gensim.models.Word2Vec(data, min_count = 1, size = 100, window = 5)
print("Cosine similarity between 'работа' " + "and 'вакансия' - CBOW : ", model1.similarity('работа', 'вакансия')) 
print("Cosine similarity between 'работа' " + "and 'специальность' - CBOW : ", model1.similarity('работа', 'специальность')) 

Cosine similarity between 'работа' and 'вакансия' - CBOW :  0.30161306
Cosine similarity between 'работа' and 'специальность' - CBOW :  -0.09270092
Wall time: 10min 20s


In [11]:
model1.wv.save_word2vec_format('word2vec_cbow_job_description.bin', binary=True)
#model1 = KeyedVectors.load_word2vec_format('word2vec_cbow_job_description.bin', binary=True)

In [12]:
pprint(model1.most_similar('pandas', topn=10))

[('numpy', 0.9758040308952332),
 ('matplotlib', 0.9544049501419067),
 ('scipy', 0.9390780925750732),
 ('scikit-learn', 0.9363783597946167),
 ('sklearn', 0.9210339784622192),
 ('tensorflow', 0.8885713219642639),
 ('opencv', 0.8642884492874146),
 ('keras', 0.8523848056793213),
 ('perl', 0.8484242558479309),
 ('spark', 0.8437123894691467)]


In [8]:
print(f"pandas vs python: {model1.similarity('pandas', 'python'):1f}")
print(f"pandas vs java:   {model1.similarity('pandas', 'java'):1f}")

pandas vs python: 0.830975
pandas vs java:   0.646774


In [9]:
pprint(model1.most_similar(positive=['аналитик'], topn=5))

[('бизнес-аналитик', 0.7974302768707275),
 ('архитектор', 0.7880245447158813),
 ('экономист', 0.7717010974884033),
 ('маркетолог', 0.7412537932395935),
 ('интернет-маркетолог', 0.7326059341430664)]


In [10]:
print('аналитик + разработчик')
pprint(model1.most_similar(positive=['аналитик', 'разработчик'], topn=5))

аналитик + разработчик
[('архитектор', 0.8368034362792969),
 ('программист', 0.8095813989639282),
 ('тестировщик', 0.7977092266082764),
 ('ведущий', 0.7747888565063477),
 ('бизнес-аналитик', 0.7725928425788879)]


In [12]:
print('аналитик - разработчик')
pprint(model1.most_similar(positive=['аналитик'], negative=['разработчик'],topn=5))

аналитик - разработчик
[('статистика', 0.5632426738739014),
 ('ad-hoc', 0.5117810964584351),
 ('аналитика', 0.47225406765937805),
 ('план/факт', 0.47186529636383057),
 ('отчётность', 0.46078425645828247)]


## Skip Gram

In [13]:
%%time
model2 = gensim.models.Word2Vec(data, min_count = 1, size = 100, window = 5, sg = 1)
print("Cosine similarity between 'работа' " +"and 'вакансия' - Skip Gram : ", model2.similarity('работа', 'вакансия')) 
print("Cosine similarity between 'работа' " +"and 'специальность' - Skip Gram : ", model2.similarity('работа', 'специальность')) 

Cosine similarity between 'работа' and 'вакансия' - Skip Gram :  0.42258897
Cosine similarity between 'работа' and 'специальность' - Skip Gram :  0.12858261
Wall time: 36min 55s


In [16]:
model2.wv.save_word2vec_format('word2vec_skip_gram_job_description.bin', binary=True)
#model2 = KeyedVectors.load_word2vec_format('word2vec_skip_gram_job_description.bin', binary=True)

In [17]:
model2.most_similar('pandas', topn=10)

[('numpy', 0.9613885879516602),
 ('matplotlib', 0.9432936906814575),
 ('scikit-learn', 0.9337726831436157),
 ('scipy', 0.9094074964523315),
 ('scikit', 0.9082273244857788),
 ('sklearn', 0.9056084156036377),
 ('statsmodels', 0.9012402296066284),
 ('keras', 0.9005840420722961),
 ('tensorflow', 0.9000375270843506),
 ('h5py', 0.8990463018417358)]

In [16]:
print(f"pandas vs python: {model2.similarity('pandas', 'python'):1f}")
print(f"pandas vs java:   {model2.similarity('pandas', 'java'):1f}")

pandas vs python: 0.755326
pandas vs java:   0.640595


In [17]:
pprint(model2.most_similar(positive=['аналитик'], topn=5))

[('бизнес-аналитик', 0.7903444766998291),
 ('координатор', 0.7282258868217468),
 ('портфельный', 0.7216520309448242),
 ('экономист-бухгалтер', 0.7186049222946167),
 ('ассистент', 0.7174388766288757)]


In [18]:
print('аналитик + разработчик')
pprint(model2.most_similar(positive=['аналитик', 'разработчик'], topn=5))

аналитик + разработчик
[('ведущий', 0.796492338180542),
 ('программист', 0.7550936937332153),
 ('бизнес-аналитик', 0.7501971125602722),
 ('java-разработчик', 0.749855637550354),
 ('старший/ведущий', 0.7481011152267456)]


In [19]:
print('аналитик - разработчик')
pprint(model2.most_similar(positive=['аналитик'], negative=['разработчик'],topn=5))

аналитик - разработчик
[('воронки', 0.4527910351753235),
 ('аналитика', 0.4170919954776764),
 ('риск-менеджмент', 0.40561819076538086),
 ('аналитике', 0.3995368480682373),
 ('каналу', 0.3971827030181885)]
